In [7]:
import pandas as pd 
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
import calendar 
import win32com.client as win32
import os
import sys
import xlsxwriter
import numpy as np

# --- Configuração dos Módulos ---
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader

In [8]:
ENVIAR_PARA_TODOS = True
MODO_TESTE = True # Se True, não envia e-mail, apenas gera arquivos e printa

# Coloque aqui os nomes dos assessores
DESTINATARIO = ['FERNANDO DOMINGUES DA SILVA',
#'PAULO ROBERTO FARIA SILVA',
#'SAADALLAH JOSE ASSAD',
#'RODRIGO DE MELLO D’ELIA',
#'ROSANA APARECIDA PAVANI DA SILVA',
#'GABRIEL GUERRERO TORRES FONSECA',
#'CAIC ZEM GOMES',
#'RENAN BENTO DA SILVA',
#'RAFAEL PASOLD MEDEIROS',
#'FELIPE AUGUSTO CARDOSO',
#'MARCOS SOARES PEREIRA FILHO',
#'IZADORA VILLELA FREITAS',
#'GUILHERME DE LUCCA BERTELONI',
#'VITOR OLIVEIRA DOS REIS',
]

SCHEMA_DEFAULT = "dbo"
CAMINHO_EXPORT_EXCEL = r"C:\Scripts\liquidez"

In [9]:
try:
    basebtg = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
    times = AzureLoader.ler_tabela("times_nova_empresa", schema=SCHEMA_DEFAULT)
    saldo = AzureLoader.ler_tabela("saldo_conta_corrente", schema=SCHEMA_DEFAULT)
    posicoes = AzureLoader.ler_tabela("posicao", schema=SCHEMA_DEFAULT)
    rf_coe = AzureLoader.ler_tabela("renda_fixa", schema=SCHEMA_DEFAULT)
    proventos = AzureLoader.ler_tabela("proventos_futuros_rf", schema=SCHEMA_DEFAULT)
    
    # Carrega nomes do banco (garante que está usando a versão mais recente)
    nomes_clientes = AzureLoader.ler_tabela("nomes_clientes", schema=SCHEMA_DEFAULT)
    
    # Padronização de chaves para Merge
    for df in [basebtg, saldo, posicoes, rf_coe, proventos, nomes_clientes]:
        if 'Conta' in df.columns:
            df['Conta'] = df['Conta'].astype(str).str.strip()
        if 'CONTA' in df.columns:
            df['CONTA'] = df['CONTA'].astype(str).str.strip()
            
    print("   -> Tabelas carregadas com sucesso.")

except Exception as e:
    print(f"Erro Crítico ao carregar dados: {e}")
    sys.exit()

[AzureLoader] Lendo: dbo.base_btg...
[AzureLoader] Lendo: dbo.times_nova_empresa...
[AzureLoader] Lendo: dbo.saldo_conta_corrente...
[AzureLoader] Lendo: dbo.posicao...
[AzureLoader] Lendo: dbo.renda_fixa...
[AzureLoader] Lendo: dbo.proventos_futuros_rf...
[AzureLoader] Lendo: dbo.nomes_clientes...
   -> Tabelas carregadas com sucesso.


In [10]:
# Merge Base + Times
basebtg['Assessor'] = basebtg['Assessor'].astype(str).str.upper()
times['Assessor'] = times['Assessor'].astype(str).str.upper()
basebtg = basebtg.merge(times, on='Assessor', how='left')

# Base Reduzida
basebtg = basebtg[['Conta', 'Assessor', 'Tipo', 'TIME', 'Aniversário']]

# Merge Nomes
if 'Nome' not in basebtg.columns: # Evita duplicar se já vier do banco
    basebtg = basebtg.merge(nomes_clientes[['Conta', 'Nome']], on='Conta', how='left')

# Merge Saldo CC
saldo.rename(columns={"CONTA":"Conta"}, inplace=True)
table = pd.merge(basebtg, saldo[['Conta', 'SALDO']], on='Conta', how='left')

# Padronização Posições
posicoes.columns = posicoes.columns.str.upper()
posicoes['CONTA'] = posicoes['CONTA'].astype(str).str.strip()

table.rename(columns={"Conta":"CONTA"}, inplace=True)

# --- Cálculo de Liquidez ---

# 1. CDB Plus
cdb_plus_btg = posicoes[posicoes['PRODUTO'] == 'BTG CDB Plus FIRF CrPr']
if not cdb_plus_btg.empty:
    cdb_plus_btg = cdb_plus_btg.iloc[:,[0, 16]].groupby("CONTA").sum().reset_index()
    cdb_plus_btg.rename(columns={"VALOR LÍQUIDO":'CDB Plus'}, inplace=True)
    table = table.merge(cdb_plus_btg[['CONTA', 'CDB Plus']], on='CONTA', how='left')
else:
    table['CDB Plus'] = 0

# 2. LFTs
lft = posicoes[posicoes['PRODUTO'] == 'LFT']
if not lft.empty:
    lft = lft.iloc[:,[0, 16]].groupby("CONTA").sum().reset_index()
    lft.rename(columns={"VALOR LÍQUIDO":'LFT'}, inplace=True)
    table = table.merge(lft[['CONTA', 'LFT']], on='CONTA', how='left')
else:
    table['LFT'] = 0

# 3. CDBs de Liquidez Diária
cdb = posicoes[posicoes['PRODUTO'] == "CDB"]
rf_coe.columns = rf_coe.columns.str.upper()
rf_coe = rf_coe[['ATIVO', 'LIQUIDEZ']]

cdb = cdb.merge(rf_coe, on='ATIVO', how='left')
cdb_liq_diaria = cdb[cdb['LIQUIDEZ'] == "Liquidez Diaria"]

if not cdb_liq_diaria.empty:
    cdb_liq_diaria = cdb_liq_diaria[['CONTA', 'VALOR LÍQUIDO']].groupby("CONTA").sum().reset_index()
    cdb_liq_diaria.rename(columns={"VALOR LÍQUIDO":"CDB Líquidez Diária"}, inplace=True)
    table = table.merge(cdb_liq_diaria, on='CONTA', how='left')
else:
    table['CDB Líquidez Diária'] = 0

# 4. Vencimentos no Mês Atual
hoje = datetime.today()
res = calendar.monthrange(hoje.year, hoje.month)
ultimo_dia_do_mes = datetime(hoje.year, hoje.month, res[1]).strftime("%Y-%m-%d")

# Garante formato de data na posicao
posicoes['VENCIMENTO'] = pd.to_datetime(posicoes['VENCIMENTO'], errors='coerce')

vencimento = posicoes[(posicoes['VENCIMENTO'] <= ultimo_dia_do_mes) & (posicoes['MERCADO'] != "Valor em Trânsito")]
if not vencimento.empty:
    vencimento = vencimento[['CONTA', 'VALOR LÍQUIDO']].groupby("CONTA").sum().reset_index()
    vencimento.rename(columns={"VALOR LÍQUIDO":"Vencimentos Mes Atual"}, inplace=True)
    table = table.merge(vencimento, on='CONTA', how='left')
else:
    table['Vencimentos Mes Atual'] = 0

# Preenche NAs parciais
table.fillna(0, inplace=True)

# 5. Share of Wallet e PL Total
# Recarrega base_btg original para pegar PL (pois filtramos colunas lá em cima)
base_pl = AzureLoader.ler_tabela("base_btg", schema=SCHEMA_DEFAULT)
base_pl = base_pl[['Conta', 'PL Total', 'PL Declarado']]
base_pl.rename(columns={"Conta":"CONTA"}, inplace=True)
base_pl['CONTA'] = base_pl['CONTA'].astype(str).str.strip()
base_pl['% share of wallet'] = base_pl['PL Total']/base_pl['PL Declarado']

table = table.merge(base_pl[['CONTA', 'PL Total', "% share of wallet"]], on='CONTA', how='left')

# 6. Proventos no Mês
proventos.rename(columns={"Mes":"Data", "Conta":"CONTA"}, inplace=True)
# Agrupa se houver múltiplas linhas
proventos_por_mes = proventos.groupby(["CONTA", "Data"])["Total proventos"].sum().reset_index()

mes_atual_str = datetime.today().strftime("%Y-%m")
proventos_no_mes = proventos_por_mes[proventos_por_mes['Data'] == mes_atual_str].copy()
proventos_no_mes.rename(columns={"Total proventos":"Proventos no Mês"}, inplace=True)

table = table.merge(proventos_no_mes[['CONTA', 'Proventos no Mês']], on='CONTA', how='left')
table['Proventos no Mês'].fillna(0, inplace=True)

# 7. Soma Final
table['Soma Liquidez'] = table['SALDO'] + table['LFT'] + table['CDB Líquidez Diária'] + table['Vencimentos Mes Atual'] + table['Proventos no Mês']

# Cálculo de Liquidez (Pode gerar Infinito se PL for 0)
table["Líquidez da Carteira"] = table["Soma Liquidez"] / table["PL Total"]

print("\n[Sanitização] Tratando números infinitos e nulos...")

# 1. Substitui Infinito (divisão por zero) por 0
table.replace([np.inf, -np.inf], 0, inplace=True)

# 2. Garante que colunas numéricas sejam float e não texto
colunas_numericas = [
    'SALDO', 'CDB Plus', 'LFT', 'CDB Líquidez Diária', 'Vencimentos Mes Atual',
    'PL Total', 'Proventos no Mês', 'Soma Liquidez', 'Líquidez da Carteira',
    '% share of wallet'
]

for col in colunas_numericas:
    if col in table.columns:
        # Converte para numérico, transformando erros (textos vazios) em NaN
        table[col] = pd.to_numeric(table[col], errors='coerce')
        # Preenche NaN com 0.0
        table[col] = table[col].fillna(0.0)

[AzureLoader] Lendo: dbo.base_btg...

[Sanitização] Tratando números infinitos e nulos...


C:\Users\joaob\AppData\Local\Temp\ipykernel_30400\3901251844.py:97: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  table['Proventos no Mês'].fillna(0, inplace=True)


In [11]:
# 1. Salva no Azure (Tabela Completa)
AzureLoader.enviar_df(
    df=table, 
    nome_tabela="liquidez_clientes", 
    if_exists='replace', 
    schema=SCHEMA_DEFAULT
)

# 2. Salva Pacote Excel Local (Para backup/uso geral)
if not os.path.exists(CAMINHO_EXPORT_EXCEL):
    os.makedirs(CAMINHO_EXPORT_EXCEL)
    
table.to_excel(os.path.join(CAMINHO_EXPORT_EXCEL, "pacote_liquidez.xlsx"), index=False)

# 3. Prepara e Salva DataFrame 'infos' (Resumo para os e-mails)
infos = table[[ 
    'CONTA', 'Assessor', 'Nome', 'SALDO','PL Total', 'CDB Líquidez Diária', 'LFT', 'CDB Plus',
    'Vencimentos Mes Atual', 'Proventos no Mês', 'Soma Liquidez',
    'Líquidez da Carteira'
]]

infos.to_excel(os.path.join(CAMINHO_EXPORT_EXCEL, "infos_todos.xlsx"), index=False)

print("   -> Dados salvos no Azure e em Excel com sucesso.")

[AzureLoader] Subindo tabela: dbo.liquidez_clientes (1244 linhas)...
[AzureLoader] Chunksize calculado: 130 linhas/lote (Colunas: 16).
[AzureLoader] Concluido: liquidez_clientes atualizada.
   -> Dados salvos no Azure e em Excel com sucesso.


In [13]:
# 1. Definir datas alvo 
hoje_dt = datetime.today()
dia_semana = hoje_dt.weekday()
datas_alvo = []

if dia_semana == 4: # Sexta-feira
    print("   -> Hoje é Sexta-feira. Buscando aniversariantes de Sex, Sáb e Dom.")
    datas_alvo.append((hoje_dt.day, hoje_dt.month)) # Sexta
    sabado = hoje_dt + timedelta(days=1)
    datas_alvo.append((sabado.day, sabado.month))   # Sabado
    domingo = hoje_dt + timedelta(days=2)
    datas_alvo.append((domingo.day, domingo.month)) # Domingo
else:
    print("   -> Dia de semana comum. Buscando aniversariantes de hoje.")
    datas_alvo.append((hoje_dt.day, hoje_dt.month))

# Prepara DF de Aniversário
df_niver = basebtg[['Conta', 'Assessor', 'Tipo', 'Nome', 'Aniversário']].rename(columns={'CONTA':'Conta'}).copy()
df_niver['Aniversário'] = pd.to_datetime(df_niver['Aniversário'], errors='coerce')
df_niver = df_niver.dropna(subset=['Aniversário'])
df_niver = df_niver[df_niver['Tipo'] == 'PF'] # Apenas PF

df_niver['Dia'] = df_niver['Aniversário'].dt.day
df_niver['Mes'] = df_niver['Aniversário'].dt.month

   -> Hoje é Sexta-feira. Buscando aniversariantes de Sex, Sáb e Dom.


In [14]:
try:
    outlook = win32.Dispatch('outlook.application')
except:
    print("   [AVISO] Outlook não detectado. Pule esta etapa se estiver rodando no servidor sem Outlook.")
    outlook = None

if ENVIAR_PARA_TODOS:
    lista_assessores = times['Assessor'].unique()
else:
    lista_assessores = DESTINATARIO

count_envios = 0

if outlook:
    for nome_assessor in lista_assessores:
        nome_assessor = str(nome_assessor).upper()
        
        # Busca email do assessor na tabela times
        dados_assessor = times[times['Assessor'] == nome_assessor]
        
        if dados_assessor.empty:
            # Tenta um "match" parcial se o nome exato falhar
            # (Opcional, removi para manter a lógica estrita e segura do seu código original)
            continue 
        
        # Regras de Destinatário (Hardcoded conforme seu original)
        if nome_assessor == 'FERNANDO DOMINGUES DA SILVA':
            email_destino = 'murilo.gino@atriacm.com.br; leonardo.demetrio@atriacm.com.br'
        elif nome_assessor == 'PAULO ROBERTO FARIA SILVA':
            email_destino = 'paulo.faria@atriacm.com.br; leonardo.demetrio@atriacm.com.br'
        else:
            email_destino = dados_assessor['Email'].iloc[0]
        
        # Filtra dados de liquidez para o assessor
        infos_filtrada = infos[infos['Assessor'] == nome_assessor].copy()
        
        if infos_filtrada.empty:
            continue

        # Limpeza para o Excel
        infos_filtrada = infos_filtrada.drop(columns=['Assessor'])
        if 'SALDO' in infos_filtrada.columns:
            infos_filtrada = infos_filtrada.sort_values(by='SALDO', ascending=False)
        
        if nome_assessor == 'CAIC ZEM GOMES':
            infos_filtrada = infos_filtrada.iloc[:, :4]

        # Salva arquivo Excel individual
        nome_arquivo = f"infos_{nome_assessor.replace(' ', '_')}.xlsx"
        caminho_completo = os.path.join(CAMINHO_EXPORT_EXCEL, nome_arquivo)
        
        try:
            with pd.ExcelWriter(caminho_completo, engine='xlsxwriter') as writer:
                infos_filtrada.to_excel(writer, index=False, sheet_name='Relatorio')
                workbook = writer.book
                worksheet = writer.sheets['Relatorio']
                max_row = len(infos_filtrada)
                max_col = infos_filtrada.shape[1] - 1
                worksheet.autofilter(0, 0, max_row, max_col)
                fmt_moeda = workbook.add_format({'num_format': 'R$ #,##0.00'})   
                fmt_porcentagem = workbook.add_format({'num_format': '0.00%'})
                worksheet.set_column('A:Z', 23) 
                worksheet.set_column('B:B', 40) 
                worksheet.set_column('A:A', 14) 
                # Ajuste de colunas de moeda (C até J, verifique se bate com seu layout)
                worksheet.set_column('C:J', 23, fmt_moeda) 
                worksheet.set_column('K:K', 23, fmt_porcentagem)
        except Exception as e:
            print(f"   [ERRO] Ao salvar Excel de {nome_assessor}: {e}")
            continue

        # --- HTML de Aniversariantes ---
        niver_assessor = df_niver[df_niver['Assessor'] == nome_assessor]
        lista_html_niver = ""
        tem_aniversariante = False

        if not niver_assessor.empty:
            # Filtro eficiente de datas
            filtro_data = niver_assessor.apply(lambda x: (x['Dia'], x['Mes']) in datas_alvo, axis=1)
            aniversariantes_hoje = niver_assessor[filtro_data].copy()

            if not aniversariantes_hoje.empty:
                tem_aniversariante = True
                aniversariantes_hoje.sort_values(by=['Mes', 'Dia'], inplace=True)
                
                lista_html_niver += """
                <div style="margin-top: 20px; padding: 15px; background-color: #f9f9f9; border-left: 5px solid #2C3E50; font-family: Arial, sans-serif;">
                <h3 style="color: #2C3E50; margin-top: 0;">Aniversariantes</h3>
                <table style="border-collapse: collapse; width: 100%;">
                    <tr style="background-color: #e0e0e0; font-size: 12px;">
                        <th style="padding: 5px; text-align: left;">Data</th>
                        <th style="padding: 5px; text-align: left;">Conta</th>
                        <th style="padding: 5px; text-align: left;">Cliente</th>
                    </tr>
                """
                
                for idx, row in aniversariantes_hoje.iterrows():
                    dia_nasc = f"{row['Dia']:02d}/{row['Mes']:02d}"
                    nome_cli = row['Nome']
                    conta_cli = row['Conta']
                    
                    label_dia = ""
                    if dia_semana == 4:
                        if row['Dia'] == hoje_dt.day: label_dia = " (Hoje)"
                        elif row['Dia'] == (hoje_dt + timedelta(days=1)).day: label_dia = " (Sáb)"
                        elif row['Dia'] == (hoje_dt + timedelta(days=2)).day: label_dia = " (Dom)"

                    lista_html_niver += f"""
                    <tr style="border-bottom: 1px solid #ddd;">
                        <td style="padding: 8px; color: #555; white-space: nowrap;"><b>{dia_nasc}</b><span style="font-size:10px; color:#777">{label_dia}</span></td>
                        <td style="padding: 8px; color: #333;">{conta_cli}</td>
                        <td style="padding: 8px; color: #333;">{nome_cli}</td>
                    </tr>
                    """
                lista_html_niver += "</table></div>"

        # --- Envio do E-mail ---
        primeiro_nome = nome_assessor.split()[0].title()
        
        if MODO_TESTE:
            print(f"--- [TESTE] Arquivo: {nome_arquivo} | Para: {email_destino}")
            if tem_aniversariante:
                print("    [TESTE] > Inclui tabela de aniversariantes.")
        else:
            try:
                email = outlook.CreateItem(0)
                email.To = email_destino
                email.Subject = "Relatório de Saldo, Liquidez e Aniversariantes"
                
                email.HTMLBody = f"""
                <div style="font-family: Arial, sans-serif; color: #333;">
                    <p>Olá, {primeiro_nome}</p>
                    <p>Segue em anexo a planilha formatada com os saldos das contas atualizados.</p>
                    
                    {lista_html_niver}
                    
                    <br>
                    <p>Qualquer dúvida ou problema estou a disposição!</p>
                    <p>Abs,</p>
                </div>
                """
                email.Attachments.Add(caminho_completo)
                email.Send()
                print(f"[SUCESSO] Enviado para: {nome_assessor}")
                count_envios += 1
            except Exception as e:
                print(f"[FALHA] Erro ao enviar para {nome_assessor}: {e}")

print(f"\nProcessamento concluído. {count_envios} e-mails enviados.")

--- [TESTE] Arquivo: infos_CAIC_ZEM_GOMES.xlsx | Para: caic.zem@atriacm.com.br
--- [TESTE] Arquivo: infos_FELIPE_AUGUSTO_CARDOSO.xlsx | Para: felipe.cardoso@atriacm.com.br
--- [TESTE] Arquivo: infos_FERNANDO_DOMINGUES_DA_SILVA.xlsx | Para: murilo.gino@atriacm.com.br; leonardo.demetrio@atriacm.com.br
    [TESTE] > Inclui tabela de aniversariantes.
--- [TESTE] Arquivo: infos_GUILHERME_DE_LUCCA_BERTELONI.xlsx | Para: guilherme.delucca@atriacm.com.br
--- [TESTE] Arquivo: infos_IZADORA_VILLELA_FREITAS.xlsx | Para: izadora.villela@atriacm.com.br
    [TESTE] > Inclui tabela de aniversariantes.
--- [TESTE] Arquivo: infos_MARCOS_SOARES_PEREIRA_FILHO.xlsx | Para: marcos.soares@atriacm.com.br
--- [TESTE] Arquivo: infos_PAULO_ROBERTO_FARIA_SILVA.xlsx | Para: paulo.faria@atriacm.com.br; leonardo.demetrio@atriacm.com.br
--- [TESTE] Arquivo: infos_RAFAEL_PASOLD_MEDEIROS.xlsx | Para: rafael.pasold@atriacm.com.br
--- [TESTE] Arquivo: infos_RENAN_BENTO_DA_SILVA.xlsx | Para: renan.bento@atriacm.com.br
--